In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException
import time
import pandas as pd

In [39]:
# 크롬 옵션즈에 정보를 담아 사람인 것 처럼 만들기
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# 크롬 웹브라우저 드라이버 자동 다운로드
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
driver.set_window_size(1920,1080) # 웹브라우저 해상도 조절
driver.get("https://map.kakao.com/")

In [40]:
try:
    # 해당 요소 찾기
    coach_layer_image = driver.find_element(By.CSS_SELECTOR, "body > div.coach_layer.coach_layer_type1 > div > div > img")
    # JavaScript를 사용해 클릭
    driver.execute_script("arguments[0].click();", coach_layer_image)
    print("코치 레이어 이미지를 클릭했습니다.")
except NoSuchElementException:
    # 요소가 없으면 무시하고 진행
    print("코치 레이어 이미지가 존재하지 않습니다. 다음 단계로 진행합니다.")


코치 레이어 이미지를 클릭했습니다.


In [41]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# 검색창 찾기
search_box = driver.find_element(By.ID, "search.keyword.query")
search_box.send_keys("한강수")  # 원하는 검색어 입력
search_box.send_keys(Keys.ENTER)  # 엔터 키로 검색 실행

In [42]:
# 잠시 대기 후 첫 번째 검색 결과 클릭
time.sleep(1)  # 검색 결과가 로딩될 시간을 줌
first_result = driver.find_element(By.CSS_SELECTOR, '.placelist .PlaceItem .link_name')  # 첫 번째 항목의 선택자
first_result.click()

In [43]:
# 첫 번째 검색 결과의 리뷰 링크 클릭
moreview_link = driver.find_element(By.CSS_SELECTOR, 'a[data-id="numberofscore"]')
moreview_link.click()

In [44]:
original_window = driver.current_window_handle
for window_handle in driver.window_handles:
    if window_handle != original_window:
        driver.switch_to.window(window_handle)
        break

In [45]:
from selenium.common.exceptions import TimeoutException

# '후기 더보기' 버튼을 반복적으로 클릭하여 모든 리뷰를 로드
while True:
    try:
        # '후기 더보기' 버튼이 있는지 확인
        more_reviews_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="mArticle"]/div[7]/div[3]/a'))
        )
        
        # 버튼의 텍스트가 '후기 접기'인지 확인하여 모든 리뷰가 로드되었는지 확인
        if more_reviews_button.text == "후기 접기":
            print("모든 리뷰가 로드되었습니다.")
            break
        
        # '후기 더보기' 버튼 클릭
        driver.execute_script("arguments[0].click();", more_reviews_button)
        print("후기 더보기 버튼을 클릭했습니다.")
        
        # 리뷰가 로드되는 동안 약간의 대기 시간을 추가
        time.sleep(1)

    except TimeoutException:
        # '후기 더보기' 버튼이 없으면 루프 종료
        print("후기 더보기 버튼을 찾을 수 없거나 로드되지 않았습니다. 모든 리뷰가 이미 로드되었거나 버튼이 없습니다.")
        break

    except ElementClickInterceptedException:
        # 버튼이 클릭되지 않는 경우 메시지 출력 후 루프 종료
        print("후기 더보기 버튼을 클릭할 수 없습니다. 다른 요소가 가로막고 있을 수 있습니다.")
        break

print("모든 리뷰 로드 완료.")


후기 더보기 버튼을 클릭했습니다.
후기 더보기 버튼을 클릭했습니다.
후기 더보기 버튼을 클릭했습니다.
모든 리뷰가 로드되었습니다.
모든 리뷰 로드 완료.


In [46]:
# 모든 리뷰를 담을 리스트 초기화
review_dates = []
review_contents = []

# 리뷰 목록 가져오기
review_elements = driver.find_elements(By.CSS_SELECTOR, 'ul.list_evaluation li')

# 각 리뷰에서 날짜와 내용 추출
for review_element in review_elements:
    try:
        # 리뷰 날짜와 내용 추출
        date = review_element.find_element(By.CSS_SELECTOR, '.unit_info .time_write').text
        content = review_element.find_element(By.CSS_SELECTOR, '.txt_comment').text
        
        # 리스트에 저장
        review_dates.append(date)
        review_contents.append(content)
        
    except Exception as e:
        print("리뷰 정보를 가져오는 중 문제가 발생했습니다:", e)

# 데이터프레임으로 저장
reviews_df = pd.DataFrame({
    '작성 날짜': review_dates,
    '리뷰 내용': review_contents
})

# 데이터프레임 출력
print(reviews_df)

리뷰 정보를 가져오는 중 문제가 발생했습니다: Message: no such element: Unable to locate element: {"method":"css selector","selector":".unit_info .time_write"}
  (Session info: chrome=130.0.6723.92); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0098EBD3+24307]
	(No symbol) [0x00918D74]
	(No symbol) [0x007FC323]
	(No symbol) [0x0083DC86]
	(No symbol) [0x0083DECB]
	(No symbol) [0x00833CF1]
	(No symbol) [0x0085FED4]
	(No symbol) [0x00833C14]
	(No symbol) [0x00860124]
	(No symbol) [0x00879579]
	(No symbol) [0x0085FC26]
	(No symbol) [0x0083219C]
	(No symbol) [0x0083311D]
	GetHandleVerifier [0x00C38D93+2818227]
	GetHandleVerifier [0x00C9542E+3196750]
	GetHandleVerifier [0x00C8D9D2+3165426]
	GetHandleVerifier [0x00A2DA70+675216]
	(No symbol) [0x00921B3D]
	(No symbol) [0x0091EA18]
	(No symbol) [0x0091EBB5]
	(No symbol) [0x00911640]
	BaseThreadInitThunk [0x7712FCC9+25]
	RtlGetAppCon

리뷰 정보를 가져오는 중 문제가 발생했습니다: Message: no such element: Unable to locate element: {"method":"css selector","selector":".unit_info .time_write"}
  (Session info: chrome=130.0.6723.92); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0098EBD3+24307]
	(No symbol) [0x00918D74]
	(No symbol) [0x007FC323]
	(No symbol) [0x0083DC86]
	(No symbol) [0x0083DECB]
	(No symbol) [0x00833CF1]
	(No symbol) [0x0085FED4]
	(No symbol) [0x00833C14]
	(No symbol) [0x00860124]
	(No symbol) [0x00879579]
	(No symbol) [0x0085FC26]
	(No symbol) [0x0083219C]
	(No symbol) [0x0083311D]
	GetHandleVerifier [0x00C38D93+2818227]
	GetHandleVerifier [0x00C9542E+3196750]
	GetHandleVerifier [0x00C8D9D2+3165426]
	GetHandleVerifier [0x00A2DA70+675216]
	(No symbol) [0x00921B3D]
	(No symbol) [0x0091EA18]
	(No symbol) [0x0091EBB5]
	(No symbol) [0x00911640]
	BaseThreadInitThunk [0x7712FCC9+25]
	RtlGetAppCon

리뷰 정보를 가져오는 중 문제가 발생했습니다: Message: no such element: Unable to locate element: {"method":"css selector","selector":".unit_info .time_write"}
  (Session info: chrome=130.0.6723.92); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0098EBD3+24307]
	(No symbol) [0x00918D74]
	(No symbol) [0x007FC323]
	(No symbol) [0x0083DC86]
	(No symbol) [0x0083DECB]
	(No symbol) [0x00833CF1]
	(No symbol) [0x0085FED4]
	(No symbol) [0x00833C14]
	(No symbol) [0x00860124]
	(No symbol) [0x00879579]
	(No symbol) [0x0085FC26]
	(No symbol) [0x0083219C]
	(No symbol) [0x0083311D]
	GetHandleVerifier [0x00C38D93+2818227]
	GetHandleVerifier [0x00C9542E+3196750]
	GetHandleVerifier [0x00C8D9D2+3165426]
	GetHandleVerifier [0x00A2DA70+675216]
	(No symbol) [0x00921B3D]
	(No symbol) [0x0091EA18]
	(No symbol) [0x0091EBB5]
	(No symbol) [0x00911640]
	BaseThreadInitThunk [0x7712FCC9+25]
	RtlGetAppCon

In [47]:
reviews_df

,작성 날짜,리뷰 내용
0,2024.10.21.,좋은 모임 자리를 갖게 되어서 기분 좋은 자리 가지고 싶어서 서버분 팁도 먼저 드리...
1,2024.09.11.,
2,2024.02.23.,"가성비가 좋은 건 인정하고 맛도 그럭저럭 좋다. 다만, 룸에서는 일부 메뉴 (등급낮..."
3,2024.01.07.,"주로 따로국밥, 우거지탕과 같은 식사류를 먹어봤는데 가격대비 퀄리티가 너무 좋다. ..."
4,2023.10.06.,
5,2023.09.20.,
6,2023.09.04.,
7,2023.05.21.,
8,2023.05.14.,"진짜 직원들 매너 무엇,,, 기분나빠서 나옴"
9,2023.01.03.,강남에서 높은 질의 한우를 이 가격에 먹을 수 있다는 점에 한 번 놀라고 전라도 김...


### -------------------------------------------------------------